In [1]:
# Simulating a full game based only on gameid, teamname, playername, league, champion, side, position

# This model uses ALL data from range of 2014 to 2023 to make this prediction

# This model takes in only 7 features as input and tries to predict all the rest of the 96 features.
# The output of this model would essentially be data of a full game including but not limited to;
# game result, game length, 5 champion bans, number of dragons and barons taken, etc.

# The prediction of this model is heavily influenced by the data provided.
# If you want to see the data which it uses when prediciting or to tweak it:
#   League Of Legends Esports Analysis - Group 1\Data\PredictionData\2023WorldsPredictions(ModelA).csv

# If you want to see the original data that is not modified then you can read the following
#   League Of Legends Esports Analysis - Group 1\Data\PredictionData\2023WorldsPredictions(ORIGINALDATA)

# Also MAKE SURE TO ADD IN YOUR ABSOLUTE PATH BEFORE RUNNING IN 'df_files'. Please CRTL+F and replace
# with your own abolute path to this project directory.

In [2]:
# Make Notebook cells strech to fit the browsers's full width resolution
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Importing Libraries
import pandas as pd
import numpy as np
#import tensorflow as tf
import os

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from glob import glob
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.datasets import load_iris
from sklearn.preprocessing import OrdinalEncoder

from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
#from keras.utils.vis_utils import plot_model
from sklearn.metrics import r2_score
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
import re


In [3]:
# Grab all files
# To run this add in the absolute path to the /Data/20**.csv | (this will essentially grab csv 2014 to 2023)

# If you want to see a quick example of a run use only 2023.csv, which only takes about 9 to 10 mins to run
# A full dataset run would be 20**.csv, which takes a long time!
df_files = sorted(glob("/Users/Gagan/Documents/Anaconda Projects/League Of Legends Esports Analysis - Group 1/Data/20**.csv")) # 20**.csv

# Combine all the files into one big dataframe
df_Overall = pd.concat((pd.read_csv(file, low_memory=False) for file in df_files))
print(df_files)


# Filter rows based on data completeness and position
df_Overall = df_Overall[(df_Overall['datacompleteness'] == 'complete')]#& (df_Overall['datacompleteness'] != 'team')

# Drop unnecessary columns
cols_to_drop = ['datacompleteness', 'url', 'split', 'playoffs', 'date', 'game','participantid', 'playerid', 'teamid', 'firstbloodassist', 'firstbloodkill', 'firstbloodvictim', 'dragons (type unknown)', 'damageshare', 'earnedgoldshare', 'monsterkillsownjungle', 'monsterkillsenemyjungle', 'year', 'patch', 'teamkills', 'teamdeaths']
df_Overall = df_Overall.drop(columns=cols_to_drop)

['/Users/Gagan/Documents/Anaconda Projects/League Legends Win predictor/League Legends Win predictor/Data\\2014.csv', '/Users/Gagan/Documents/Anaconda Projects/League Legends Win predictor/League Legends Win predictor/Data\\2015.csv', '/Users/Gagan/Documents/Anaconda Projects/League Legends Win predictor/League Legends Win predictor/Data\\2016.csv', '/Users/Gagan/Documents/Anaconda Projects/League Legends Win predictor/League Legends Win predictor/Data\\2017.csv', '/Users/Gagan/Documents/Anaconda Projects/League Legends Win predictor/League Legends Win predictor/Data\\2018.csv', '/Users/Gagan/Documents/Anaconda Projects/League Legends Win predictor/League Legends Win predictor/Data\\2019.csv', '/Users/Gagan/Documents/Anaconda Projects/League Legends Win predictor/League Legends Win predictor/Data\\2020.csv', '/Users/Gagan/Documents/Anaconda Projects/League Legends Win predictor/League Legends Win predictor/Data\\2021.csv', '/Users/Gagan/Documents/Anaconda Projects/League Legends Win pr

In [4]:
# Fill NaN values in specific columns
df_Overall.fillna({'ban1': 'UNKNOWN', 'ban2': 'UNKNOWN', 'ban3': 'UNKNOWN', 'ban4': 'UNKNOWN', 'ban5': 'UNKNOWN', 'firstmidtower': 0}, inplace=True)

champions = pd.concat([df_Overall['ban1'], df_Overall['ban2'], df_Overall['ban3'], df_Overall['ban4'], df_Overall['ban5']]).unique()
allChampions = df_Overall['champion'].unique()
allLeagues = df_Overall['league'].unique()
allPlayerNames = df_Overall['playername'].unique()
allTeamNames = df_Overall['teamname'].unique()

# Convert categorical values to numerical values
value_to_index = {value: index for index, value in enumerate(champions)}
value_to_index1 = {value: index for index, value in enumerate(allChampions)}
value_to_index2League = {value: index for index, value in enumerate(allLeagues)}
value_to_index3PlayerNames = {value: index for index, value in enumerate(allPlayerNames)}
value_to_index4TeamNames = {value: index for index, value in enumerate(allTeamNames)}

df_Overall['teamname'] = df_Overall['teamname'].map(value_to_index4TeamNames)
df_Overall['playername'] = df_Overall['playername'].map(value_to_index3PlayerNames)
df_Overall['champion'] = df_Overall['champion'].map(value_to_index1)
df_Overall['league'] = df_Overall['league'].map(value_to_index2League)
df_Overall['ban1'] = df_Overall['ban1'].map(value_to_index)
df_Overall['ban2'] = df_Overall['ban2'].map(value_to_index)
df_Overall['ban3'] = df_Overall['ban3'].map(value_to_index)
df_Overall['ban4'] = df_Overall['ban4'].map(value_to_index)
df_Overall['ban5'] = df_Overall['ban5'].map(value_to_index)

df_Overall['side'] = df_Overall['side'].map({'Blue': 0, 'Red': 1})

df_Overall['position'] = df_Overall['position'].map({'top': 0, 'jng': 1, 'mid': 2, 'bot': 3, 'sup': 4, 'team': 5})

extracted_data = []
for game_id in df_Overall['gameid']:
    if pd.notna(game_id):
        if '/' in game_id:
            extracted_data.append(game_id.split('/')[-1])
        elif '-' in game_id:
            extracted_data.append(game_id.replace('-', '')[1])

        elif '_' in game_id:
            if '_game_' in game_id:
                extracted_data.append(game_id.split('_game_')[1])
            else:
                extracted_data.append(game_id.replace('-', '').split('_')[1])
        else:
            extracted_data.append(game_id)
    else:
        extracted_data.append(np.nan)
        
df_Overall['gameid'] = extracted_data
df_Overall.fillna(0, inplace=True)

In [5]:
X = df_Overall.drop(df_Overall.columns.difference(['gameid','teamname', 'playername', 'league', 'champion', 'side', 'position']), axis=1)
y = df_Overall[df_Overall.columns.difference(['gameid','teamname', 'playername', 'league', 'champion', 'side', 'position'])]

columns_to_exclude_scaling = ['side', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5']
columns_to_scale = df_Overall.columns.difference(columns_to_exclude_scaling)
scaler = MinMaxScaler()
df_Overall[columns_to_scale] = scaler.fit_transform(df_Overall[columns_to_scale])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

base_regressor = DecisionTreeRegressor()
model = MultiOutputRegressor(base_regressor)
model.fit(X, y)

feature_importance = model.score(X, y)

In [6]:
# Predictions
# To run this add in the absolute path to the /Data/PredictionData/2023WorldsPredictions(ModelA).csv
df_files = "C:/Users/Gagan/Documents/Anaconda Projects/League Of Legends Esports Analysis - Group 1/Data/PredictionData/2023WorldsPredictions(ModelA).csv"
df_2023Predictions = pd.read_csv(df_files, low_memory=False)

Xprediction = df_2023Predictions.drop(columns=cols_to_drop)
Xprediction = df_2023Predictions.drop(df_2023Predictions.columns.difference(['gameid','teamname', 'playername', 'league', 'champion', 'side', 'position']), axis=1)

value_to_index2League = {value: index for index, value in enumerate(allLeagues)}
value_to_index3PlayerNames = {value: index for index, value in enumerate(allPlayerNames)}
value_to_index4TeamNames = {value: index for index, value in enumerate(allTeamNames)}

Xprediction['teamname'] = Xprediction['teamname'].map(value_to_index4TeamNames)
Xprediction['playername'] = Xprediction['playername'].map(value_to_index3PlayerNames)
Xprediction['champion'] = Xprediction['champion'].map(value_to_index1)
Xprediction['league'] = Xprediction['league'].map(value_to_index2League)

Xprediction['side'] = Xprediction['side'].map({'Blue': 0, 'Red': 1})

Xprediction['position'] = Xprediction['position'].map({'top': 0, 'jng': 1, 'mid': 2, 'bot': 3, 'sup': 4})

Xprediction['gameid'] = Xprediction['gameid'].str.extract(r'_(\d+)$')

predictions = model.predict(Xprediction)
y = df_Overall[df_Overall.columns.difference(['gameid','teamname', 'playername', 'league', 'champion', 'side', 'position'])]
predicted_df = pd.DataFrame(predictions, columns=y.columns)

value_to_index1 = {value: index for index, value in enumerate(champions)}
index_to_bans = {index: value for value, index in value_to_index1.items()}
predicted_df['ban1'] = predicted_df['ban1'].map(index_to_bans)
predicted_df['ban2'] = predicted_df['ban2'].map(index_to_bans)
predicted_df['ban3'] = predicted_df['ban3'].map(index_to_bans)
predicted_df['ban4'] = predicted_df['ban4'].map(index_to_bans)
predicted_df['ban5'] = predicted_df['ban5'].map(index_to_bans)

predicted_df = pd.DataFrame(predicted_df)
pd.set_option('display.max_columns', None)
#print(predicted_df.head)
column_names = predicted_df.columns
#print(column_names)
df_2023PredClean = pd.read_csv(df_files, low_memory=False)
df_2023PredClean = df_2023PredClean.drop(columns=cols_to_drop)


In [7]:
merged_df = pd.merge(left=df_2023PredClean, right=predicted_df, how='left', left_index = True, right_index = True)
merged_df = merged_df.drop(columns=[col for col in merged_df.columns if '_x' in col])
merged_df = merged_df.rename(columns=lambda x: x.split('_y')[0] if '_y' in x else x)

# Calculating score of model on the test data
print(model.score(X_test, y_test))

display(merged_df)

0.9686229836338313


,gameid,league,side,position,playername,teamname,champion,assists,assistsat10,assistsat15,ban1,ban2,ban3,ban4,ban5,barons,chemtechs,ckpm,clouds,controlwardsbought,csat10,csat15,csdiffat10,csdiffat15,cspm,damagemitigatedperminute,damagetakenperminute,damagetochampions,deaths,deathsat10,deathsat15,doublekills,dpm,dragons,earned gpm,earnedgold,elders,elementaldrakes,firstbaron,firstblood,firstdragon,firstherald,firstmidtower,firsttothreetowers,firsttower,gamelength,goldat10,goldat15,golddiffat10,golddiffat15,goldspent,gspd,heralds,hextechs,infernals,inhibitors,kills,killsat10,killsat15,minionkills,monsterkills,mountains,oceans,opp_assistsat10,opp_assistsat15,opp_barons,opp_csat10,opp_csat15,opp_deathsat10,opp_deathsat15,opp_dragons,opp_elders,opp_elementaldrakes,opp_goldat10,opp_goldat15,opp_heralds,opp_inhibitors,opp_killsat10,opp_killsat15,opp_towers,opp_turretplates,pentakills,quadrakills,result,team kpm,total cs,totalgold,towers,triplekills,turretplates,visionscore,vspm,wardskilled,wardsplaced,wcpm,wpm,xpat10,xpat15,xpdiffat10,xpdiffat15
0,ESPORTSTMNT02_4000000,WLDs,Red,top,TheShy,Weibo Gaming,K'Stane,4.0,0.0,0.0,Xayah,Maokai,Xayah,Nautilus,Jinx,0.0,0.0,0.8687,0.0,1.0,81.0,107.0,7.0,7.0,8.2267,1101.9259,891.8009,23784.0,4.0,1.0,1.0,1.0,587.4870,0.0,183.1430,14986.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1992.0,3663.0,6286.0,396.0,725.0,17535.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,218.0,30.0,0.0,0.0,0.0,0.0,0.0,78.0,138.0,0.0,0.0,0.0,0.0,0.0,3263.0,5300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.5991,306.0,17161.0,0.0,0.0,0.0,68.0,1.2581,9.0,10.0,0.0312,0.4029,5174.0,8017.0,-559.0,-302.0
1,ESPORTSTMNT02_4000000,WLDs,Red,jng,Weiwei,Weibo Gaming,Jarvan IV,6.0,0.0,0.0,Xayah,Maokai,Xayah,Nautilus,Jinx,0.0,0.0,0.6024,0.0,1.0,57.0,86.0,-23.0,4.0,4.1897,1170.1034,782.6121,7308.0,4.0,1.0,4.0,0.0,505.2857,0.0,299.2789,5404.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1992.0,3092.0,4148.0,-940.0,456.0,10375.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,7.0,137.0,0.0,0.0,1.0,5.0,0.0,76.0,86.0,0.0,0.0,0.0,0.0,0.0,3346.0,4383.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.5991,157.0,12390.0,0.0,0.0,0.0,60.0,1.0578,24.0,19.0,0.6325,0.5787,2781.0,5668.0,-1114.0,-1715.0
2,ESPORTSTMNT02_4000000,WLDs,Red,mid,Xiaohu,Weibo Gaming,Akali,4.0,1.0,1.0,Xayah,Maokai,Xayah,Nautilus,Jinx,1.0,0.0,0.6024,0.0,2.0,64.0,135.0,-4.0,-13.0,7.5966,561.0588,570.9339,19816.0,4.0,1.0,0.0,0.0,216.9112,0.0,244.6000,6012.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1992.0,3570.0,6791.0,501.0,2620.0,9700.0,0.0,0.0,0.0,0.0,0.0,6.0,2.0,0.0,326.0,8.0,0.0,0.0,1.0,1.0,0.0,93.0,112.0,0.0,0.0,0.0,0.0,0.0,3814.0,5697.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,1.0,1.0,0.5991,159.0,17486.0,0.0,0.0,0.0,38.0,0.7420,7.0,20.0,0.1821,0.3719,4999.0,6529.0,-313.0,-1515.0
3,ESPORTSTMNT02_4000000,WLDs,Red,bot,Light,Weibo Gaming,Kai'Sa,7.0,0.0,0.0,Xayah,Maokai,Xayah,Nautilus,Jinx,0.0,0.0,0.6024,0.0,1.0,67.0,87.0,4.0,6.0,7.1707,483.1250,359.1472,31925.0,0.0,1.0,0.0,0.0,926.8374,0.0,345.9145,13689.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1992.0,3739.0,6964.0,121.0,627.0,15975.0,0.0,0.0,0.0,0.0,2.0,4.0,1.0,0.0,297.0,24.0,0.0,0.0,0.0,2.0,0.0,76.0,124.0,1.0,1.0,0.0,0.0,0.0,3559.0,5962.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.5991,352.0,13105.0,0.0,1.0,0.0,72.0,1.4010,7.0,31.0,0.4467,0.6890,3784.0,6775.0,-782.0,-133.0
4,ESPORTSTMNT02_4000000,WLDs,Red,sup,Crisp,Weibo Gaming,Thresh,2.0,0.0,0.0,Xayah,Maokai,Xayah,Nautilus,Jinx,0.0,0.0,0.8687,0.0,21.0,2.0,11.0,30.0,15.0,0.8929,529.8603,430.1538,5434.0,3.0,0.0,0.0,0.0,111.6793,0.0,117.8007,6214.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1992.0,1953.0,3573.0,-37.0,-1045.0,8575.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,26.0,0.0,0.0,0.0,1.0,3.0,0.0,12.0,12.0,1.0,1.0,0.0,0.0,0.0,2424.0,3377.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.5991,26.0,6816.0,0.0,0.0,0.0,120.0,3.6145,13.0,78.0,0.2447,1.6927,2658.0,3381.0,-57.0,72.0
5,ESPORTSTMNT02_4000000,WLDs,Blue,top,Zeus,T1,Jayce,4.0,0.0,1.0,Alistar,Neeko,Poppy,Nocturne,Jax,0.0,0.0,0.6024,0.0,4.0,85.0,108.0,27.0,-9.0,6.9501,398.1311,402.3954,19178.0,8.0,1.0,0.0,1.0,1018.3916,0.0,326.106